![imagen](./img/python.jpg)

# SQL en Python
Podrás atacar a una base de datos SQL desde muchas plataformas/lenguajes. Por supuesto, Python es uno de ellos. Para ello **utilizaremos el módulo `sqlite3`**, que viene incluido en la suite de paquetes de Anaconda.


1. [Configuración del entorno](#1.-Configuración-del-entorno)
2. [Modelo de datos](#2.-Modelo-de-datos)
3. [Queries](#3.-Queries)

    3.1 [Primera query](#3.1-Primera-query)
    
    3.2 [SELECT](#3.2-SELECT)
    
    3.3 [LIMIT](#3.3-LIMIT)
    
    3.4 [DISTINCT](#3.4-DISTINCT)
    
    3.5 [WHERE](#3.5-WHERE)
    
    3.6 [ORDER BY](#3.6-ORDER-BY)
    
    3.7 [Agregaciones](#3.7-Agregaciones)
    
    3.8 [GROUP BY](#3.8-GROUP-BY)
    
    3.9 [JOIN](#3.9-JOIN)
    
    3.10 [VIEW y DROP](#3.10-VIEW-y-DROP)
    
4. [Errores](#4.-Errores)

5. [Resumen](#5.-Resumen)


## 1. Configuración del entorno
En este taller usaremos también *Pandas*, que es el módulo para tratamiento de datos tabular que más se usa en Python.

Importamos ambos paquetes

In [10]:
import pandas as pd
import sqlite3

Lo primero que hacemos es **establecer conexión con la base de datos**. En este taller simplemente leeremos la base de datos de un archivo, pero lo normal es que tengamos que configurar la conexión a una base de datos de la empresa. Para ello existen otras librerías como `pyodbc` en el caso de un SQLServer o `cx_Oracle`, si el servidor es Oracle.

**¡Asegurate que tienes el archivo chinook.db en la ruta donde se encuentra este Notebook!**

In [4]:
import os
os.getcwd() 

import sqlite3 

# "os" proporciona una forma de interactuar con el sistema operativo. Este módulo incluye funciones para realizar operaciones como 
#leer o cambiar el directorio actual, gestionar archivos y directorios, y obtener información del entorno del sistema operativo. 

#  getcwd(), es una función del módulo os que devuelve una cadena (string) que representa la ruta del directorio de trabajo actual. 
#"Directorio de trabajo actual" se refiere al directorio desde el cual el script de Python se está ejecutando. 


In [5]:
# Conectamos con la base de datos chinook.db
connection = sqlite3.connect("chinook.db")

# Obtenemos un cursor que utilizaremos para hacer las queries
crsr = connection.cursor() 

In [6]:
# Con esta función leemos los datos y lo pasamos a un DataFrame de Pandas
def sql_query(query):

    # Ejecuta la query
    crsr.execute(query)

    # Almacena los datos de la query 
    ans = crsr.fetchall()

    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in crsr.description]

    return pd.DataFrame(ans,columns=names)

## 2. Modelo de datos
Antes de empezar a atacar una base de datos, tendremos que saber qué hay dentro, y para ello lo mejor es ver cómo es su **modelo de datos**

![imagen](./img/chinook_data_model.png)

Puedes ver todas las tablas que hay en la base de datos con la siguiente sentencia

In [7]:
# SE VEN TODAS LAS TABLAS QUE HAY EN LA BASE DE DATOS 

res = crsr.execute("SELECT name FROM sqlite_master WHERE type='table'")
for name in res:
    print(name[0]) # LA BASE DE DATOS SE LLAMA 'sqlite_master'
                    # name ES UN COMANDO QUE HACE REFERENCIA AL NOMBRE 
                     # type ES UN COMANDO QUE HACE REFERENCIA AL TIPO DE DATO DEL DATO QUE SE ESTÁ OPERANDO  

albums
sqlite_sequence
artists
customers
employees
genres
invoices
invoice_items
media_types
playlists
playlist_track
tracks
sqlite_stat1
films


## 3. Queries
En las *queries* o consultas, escribiremos nuestra sentencia SQL para poder traernos datos de la base de datos. Las queries tienen esta pinta:

```SQL
SELECT campo1, campo2, campo3...
FROM tabla
WHERE condiciones
```

Por supuesto, hay más sentencias propias de SQL que iremos viendo a lo largo del Notebook. Fíjate que las **palabras reservadas en SQL se suelen poner en mayúsculas**, para diferenciarlas del resto. No da error si se pone de otra manera, ya que SQL **no es *case sensitive***, pero sí se suele hacer así.

### 3.1 Primera query

In [17]:
query = '''
SELECT * 
FROM tracks
'''

df = sql_query(query)
df # SELECCIONA TODOS LOS REGISTROS DESDE LA TABLA 'tracks', EN LA CELDA ANTERIOR SE PUEDE OBSERVAR QUE 'tracks' ES EL NOMBRE DE 
    #UNA BASE DE DATOS   

    # 'sql_query(query)' ES EL EQUIVALENTE AL 'return' DE LAS FUNCIONES 

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99
...,...,...,...,...,...,...,...,...,...
3498,3499,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,343,2,24,None,286741,4718950,0.99
3499,3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",344,2,24,Franz Schubert,139200,2283131,0.99
3500,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",345,2,24,Claudio Monteverdi,66639,1189062,0.99
3501,3502,"Quintet for Horn, Violin, 2 Violas, and Cello ...",346,2,24,Wolfgang Amadeus Mozart,221331,3665114,0.99


### 3.2 SELECT
**La sentencia `SELECT` se usa para filtrar columnas**. Podremos incluso renombrar algunas columnas dentro del propio `SELECT`. Dos opciones:
* **SELECT * :** de esta forma lo que haremos es traer todos los campos de la tabla. NO es lo recomendable ya que hay tablas que pueden tener cientos de columnas y probablemente no necesitemos todas, por lo que afectará al rendimiento de la maquina y en definitiva al resto de usuarios. **Recuerda que las bases de datos son normalmente entornos compartidos por varios usuarios :)**
* **SELECT campo1, campo2...:** sentencia para escoger los campos concretos que necesitamos.

Además en la propia query podrás renombrar campos mediante la sintaxis `campo as nuevo_nombre`. **Si quieres poner espacios en el nombre del campo, tendrás que rodear el string con comillas dobles**

SQL no es sensible a mayusculas y minusculas.

In [7]:
query = '''
SELECT Name AS "Nombre Cancion", composer AS "Compositor"
FROM tracks
'''

sql_query(query) # DESDE LA BASE DE DATOS 'tracks' SE SELECCIONA EL CAMPO 'Name' Y SE CAMBIA SU NOMBRE POR 'Nombre Cancion', ADEMÁS 
                  # SE SELECCIONA DESDE LA MISMA BASE DE DATOS EL CAMPO 'composer' Y SE CAMBIA SU NOMBRE POR 'Compositor' 

,Nombre Cancion,Compositor
0,For Those About To Rock (We Salute You),"Angus Young, Malcolm Young, Brian Johnson"
1,Balls to the Wall,None
2,Fast As a Shark,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho..."
3,Restless and Wild,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D..."
4,Princess of the Dawn,Deaffy & R.A. Smith-Diesel
...,...,...
3498,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,None
3499,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",Franz Schubert
3500,"L'orfeo, Act 3, Sinfonia (Orchestra)",Claudio Monteverdi
3501,"Quintet for Horn, Violin, 2 Violas, and Cello ...",Wolfgang Amadeus Mozart


### 3.3 LIMIT
Se usa para acotar el número de registros de la query. Va siempre al final. Por ejemplo `LIMIT 10`

In [8]:
query = '''
SELECT Name AS "Nombre Cancion", composer
FROM tracks
LIMIT 10
'''

sql_query(query) # DE LA TABLA 'tracks' SELECCIONA Y LE CAMBIA EL NOMBRE AL CAMPO 'Name' POR EL NOMBRE 'Nombre Cancion', Y 
                  # SELECCIONA EL CAMPO 'composer'. SELECCIONA LOS 10 PRIMEROS REGISTROS    

,Nombre Cancion,Composer
0,For Those About To Rock (We Salute You),"Angus Young, Malcolm Young, Brian Johnson"
1,Balls to the Wall,None
2,Fast As a Shark,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho..."
3,Restless and Wild,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D..."
4,Princess of the Dawn,Deaffy & R.A. Smith-Diesel
5,Put The Finger On You,"Angus Young, Malcolm Young, Brian Johnson"
6,Let's Get It Up,"Angus Young, Malcolm Young, Brian Johnson"
7,Inject The Venom,"Angus Young, Malcolm Young, Brian Johnson"
8,Snowballed,"Angus Young, Malcolm Young, Brian Johnson"
9,Evil Walks,"Angus Young, Malcolm Young, Brian Johnson"


### 3.4 DISTINCT
Se usa para obtener todos los registros únicos, es decir, sin duplicados. Muy útil tanto para eliminar dupicados, como para ver todas las casuísticas de un campo en concreto.

**Mucho cuidado con esta sentencia ya que si la tabla tiene miles o millones de registros, puede ralentizar mucho la query.**

In [9]:
query = '''
SELECT DISTINCT(Composer)
FROM tracks
'''

sql_query(query) # SELECT 'DISTINCT(Composer)' SELECCIONA TODOS LOS ELEMENTOS DEL CAMPO 'composer' SIN HACER DUPLICADO DE ESTOS 
                  # AUNQUE SI ESTÉ REPETIDO EL VALOR 

,Composer
0,"Angus Young, Malcolm Young, Brian Johnson"
1,None
2,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho..."
3,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D..."
4,Deaffy & R.A. Smith-Diesel
...,...
848,Carl Nielsen
849,Niccolò Paganini
850,Pietro Antonio Locatelli
851,Claudio Monteverdi


### 3.5 WHERE
Se usa para filtrar filas. Ejemplos de uso:
* **Un valor numérico**
    * UnitPrice = 0.99
    * UnitPrice >= 0.99
    * UnitPrice < 0.99
* **Un valor string**: Name = 'Restless and Wild'
* **Varios valores**: GenreId in (1, 5, 12)
* **string contenido**:
    * strings que empiecen por 'A': Name like 'A%'
    * strings que acaben en 'A': Name like '%A'
    * strings que lleven 'A' en algun punto: Name like '%A%'
* **Distinto de**: UnitPrice <> 0.99

Probamos el `WHERE`

In [18]:
query = ''' 
SELECT * 
FROM tracks
WHERE unitprice > 0.99
'''

sql_query(query)  # LAS COMILLAS SIMPLES SON PARA ESCRIBIR LA INFORMACIÓN DE UNA LINEA EN TRES LINEAS 

                   # DESDE LA TABLA 'tracks' SELECCIONA TODOS LOS REGISTROS DONDE EL CAMPO 'unitprice' SEA MAYOR QUE 0.99 

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,2819,Battlestar Galactica: The Story So Far,226,3,18,None,2622250,490750393,1.99
1,2820,Occupation / Precipice,227,3,19,None,5286953,1054423946,1.99
2,2821,"Exodus, Pt. 1",227,3,19,None,2621708,475079441,1.99
3,2822,"Exodus, Pt. 2",227,3,19,None,2618000,466820021,1.99
4,2823,Collaborators,227,3,19,None,2626626,483484911,1.99
...,...,...,...,...,...,...,...,...,...
208,3362,"There's No Place Like Home, Pt. 1",261,3,21,None,2609526,522919189,1.99
209,3363,"There's No Place Like Home, Pt. 2",261,3,21,None,2497956,523748920,1.99
210,3364,"There's No Place Like Home, Pt. 3",261,3,21,None,2582957,486161766,1.99
211,3428,Branch Closing,251,3,22,None,1814855,360331351,1.99


Veamos qué tal funciona el `LIKE`

In [44]:
query = '''
SELECT * 
FROM tracks
WHERE composer LIKE '%Brian Johnson%'
'''
# SELECCIONAR EN LA TABLA 'tracks' LOS REGISTROS EN QUE EL CAMPO 'composer' SEA 'Brian Johnson'
sql_query(query) 

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,6,Put The Finger On You,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",205662,6713451,0.99
2,7,Let's Get It Up,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",233926,7636561,0.99
3,8,Inject The Venom,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",210834,6852860,0.99
4,9,Snowballed,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",203102,6599424,0.99
5,10,Evil Walks,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",263497,8611245,0.99
6,11,C.O.D.,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",199836,6566314,0.99
7,12,Breaking The Rules,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",263288,8596840,0.99
8,13,Night Of The Long Knives,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",205688,6706347,0.99
9,14,Spellbound,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",270863,8817038,0.99


Compliquemos un poco más el `WHERE`

In [12]:
query = '''
SELECT * 
FROM tracks
WHERE (unitprice > 0.99 OR bytes > 100000000) AND genreid IN (21, 22, 23) AND Name LIKE '%al%'
'''
 # SELECCIONAR EN LA TABLA 'tracks' LOS REGISTROS DONDE EL CAMPO 'unitprice' SEA MAYOR QUE 0.99 Ó LOS BYTES SEAN MAYORES QUE 100000000
  # Y ADEMÁS, EL CAMPO 'genreid' SEAN 21, 22 O 23; Y QUE ADEMÁS EL CAMPO 'Name' CONTENGA LA LETRA 'a' 
df = sql_query(query)
df

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,2844,Better Halves,228,3,21,None,2573031,549353481,1.99
1,2849,Fallout,228,3,21,None,2594761,501145440,1.99
2,2862,The Glass Ballerina,229,3,21,None,2637458,535729216,1.99
3,2882,Lost Survival Guide,229,3,21,None,2632590,486675063,1.99
4,2899,The Man from Tallahassee,229,3,21,None,2637637,550893556,1.99
5,2922,"Live Together, Die Alone, Pt. 1",231,3,21,None,2478041,457364940,1.99
6,3211,Traveling Salesmen,251,3,22,None,1289039,250822697,1.99
7,3219,Product Recall,251,3,22,None,1268268,251208610,1.99


### 3.6 ORDER BY
Podemos **ordenar la tabla por el campo/s que queramos**. Por defecto ordena alfabéticamente los strings y de menor a mayor los tipos numéricos. Si quieres que ordene al revés, tienes que poner `DESC` de la forma `ORDER BY campo DESC`

In [13]:
query = '''
SELECT * 
FROM tracks
ORDER BY name DESC;
'''
# SELECCIONAR TODOS LOS CAMPOS DE LA TABLA 'tracks' Y TENIENDO COMO REFERENCIA EL CAMPO 'name' ORDENAR LOS REGISTROS DE FORMA DESCENDENTE 
sql_query(query) 

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1077,Último Pau-De-Arara,85,1,10,Corumbá/José Gumarães/Venancio,200437,6638563,0.99
1,1073,Óia Eu Aqui De Novo,85,1,10,None,219454,7469735,0.99
2,2078,Óculos,169,1,7,None,219271,7262419,0.99
3,3496,"Étude 1, In C Major - Preludio (Presto) - Liszt",340,4,24,None,51780,2229617,0.99
4,333,É que Nessa Encarnação Eu Nasci Manga,29,1,9,Lucina/Luli,196519,6568081,0.99
...,...,...,...,...,...,...,...,...,...
3498,3254,#9 Dream,255,2,9,None,278312,4506425,0.99
3499,109,#1 Zero,11,1,4,"Cornell, Commerford, Morello, Wilk",299102,9731988,0.99
3500,3412,"""Eine Kleine Nachtmusik"" Serenade In G, K. 525...",281,2,24,Wolfgang Amadeus Mozart,348971,5760129,0.99
3501,2918,"""?""",231,3,19,None,2782333,528227089,1.99


### 3.7 Agregaciones
En ocasiones nos interesa obtener algún estadístico como el máximo de un campo, su desviación estándar o simplemente un conteo de registros no nulos. Para ello podemos usar funciones como `MAX`, `COUNT` o `AVG`. En [esta página](https://www.sqlservertutorial.net/sql-server-aggregate-functions/) encontrarás un resumen con las principales funciones.

In [14]:
# Aggregations
query = '''
SELECT COUNT(*)
FROM tracks
WHERE name LIKE 'a%';
'''
 # CONTAR TODOS LOS REGISTOS DE LA TABLA 'tracks' DONDE EL CAMPO 'name' TENGA LA LETRA 'a' 
sql_query(query)

,COUNT(*)
0,199


In [15]:
query = '''
SELECT AVG(unitprice)
FROM invoice_items;
'''
 # 1- Define una consulta SQL: La variable query contiene una cadena con una consulta SQL que calcula el promedio del precio unitario de 
  #los ítems en la tabla 'invoice_items'. 

 # 2- Ejecuta la consulta SQL: La función sql_query(query) se llama con la consulta SQL como argumento. Esta función ejecuta la consulta
  # en una base de datos y devuelve el resultado.

# Ejecuta la consulta SQL: La función sql_query(query) se llama con la consulta SQL como argumento. Esta función ejecuta la consulta en
#una base de datos y devuelve el resultado.
sql_query(query)

,AVG(unitprice)
0,1.039554


### 3.8 GROUP BY
Es una sentencia muy útil para **calcular agregados en función de otro campo**. Por ejemplo, para calcular el total del precio unitario en función del género


In [16]:
query = '''
SELECT GenreId, SUM(unitprice) AS Tot_price
FROM tracks
GROUP BY GenreId
ORDER BY Tot_price DESC
LIMIT 10;
'''
 # Define una consulta SQL: La variable query contiene una cadena con una consulta SQL que calcula en la tabla 'tracks' la suma del
  #campo 'unitprice' de cada código del campo 'GenreId', cada valor de 'GenreId, SUM(unitprice)' se lo asigna al campo 'Tot_price'; y 
   #luego ordena los valores del campo 'Tot_price' de mayor a menor, devolviendo los valores del campo 'GenreId' con el valor del campo 
    #'Tot_price' más alto. 
sql_query(query)

,GenreId,Tot_price
0,1,1284.03
1,7,573.21
2,3,370.26
3,4,328.68
4,19,185.07
5,2,128.70
6,21,127.36
7,6,80.19
8,24,73.26
9,14,60.39


O calcular cuantas canciones hay por compositor

In [45]:
query = '''
SELECT composer, COUNT(trackid)
FROM tracks
WHERE Composer IS NOT NULL
GROUP BY composer
ORDER BY 2 DESC
LIMIT 100
;
'''
# Define una consulta SQL: La variable query contiene una cadena con una consulta SQL que selecciona los 100 registros del campo 
 #'Composer' cuya suma de los registros del campo 'trackid' sean las más alta de la tabla 'tracks'
 
sql_query(query) #ORDER BY 2 DESC: SIGNIFICA QUE CONSIDERE EL SEGUNDO CAMPO "COUNT(trackid)" Y ORDENE DE FORMA DESCENDENTE LAS FILAS 

,Composer,COUNT(trackid)
0,Steve Harris,80
1,U2,44
2,Jagger/Richards,35
3,Billy Corgan,31
4,Kurt Cobain,26
...,...,...
95,"James Hetfield, Lars Ulrich, Kirk Hammett",7
96,Ian Gillan/Ian Paice/Jon Lord/Ritchie Blckmore...,7
97,Harris,7
98,Green Day,7


### 3.9 JOIN
Hasta ahora hemos hecho queries sobre una única tabla, pero **¿y si queremos juntar datos de varias tablas?** Para eso están los `JOIN`s. Para ello **necesitas tener uno o varios campos comunes entre ambas tablas, que denominan CLAVES**.

**¿Cuándo usarlos?** Por ejemplo, si tenemos una tabla con un conjunto de clientes y necesitamos añadirles campos nuevos, tendremos que acudir a otras tablas donde esté ese identificador de cliente y aplicar un `JOIN`. Es lo que se conoce como *pegar campos* a otra tabla.

O imagina que tienes una tabla con todos tus pedidos, con muchos campos(ciudad, dirección, cliente...) y en otra tabla únicamente los números de pedido que no se llegaron a entregar. Si quieres filtrar dentro de tu tabla total de pedidos los que no se llegaron a entregar, podrías aplicar un `INNER JOIN` de manera que te quedes con lo común en ambas tablas, siendo tu clave el identificativo del pedido..

Existen varios tipos de JOINs:

![imagen](./img/joins.jpg) 

![imagen](./img/chinook_data_model.png)

In [19]:
#INNER JOIN 
query = '''
SELECT a.name AS track_name, a.composer, b.name AS genre_name 
FROM tracks AS a
INNER JOIN genres AS b
ON a.genreid = b.genreid
;
'''
# SELECT a.name AS track_name, a.composer, b.name AS genre_name: Selecciona el campo 'track_name' (a.name), el campo 'composer' (a.composer) 
#y genre_name (b.name), asignando alias a track_name y genre_name para mayor claridad.

#FROM tracks AS a: Especifica que los datos deben ser seleccionados de la tabla tracks con el alias a.

#INNER JOIN genres AS b: Realiza una unión interna entre la tabla tracks y la tabla genres, asignando el alias b a genres.

#ON a.genreid = b.genreid: Define la condición de la unión, donde el campo genreid de tracks debe coincidir con el campo genreid de
#genres. 

sql_query(query) # a. = tabla a y b. = tabla b 

,track_name,Composer,genre_name
0,For Those About To Rock (We Salute You),"Angus Young, Malcolm Young, Brian Johnson",Rock
1,Balls to the Wall,None,Rock
2,Fast As a Shark,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",Rock
3,Restless and Wild,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",Rock
4,Princess of the Dawn,Deaffy & R.A. Smith-Diesel,Rock
...,...,...,...
3498,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,None,Classical
3499,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",Franz Schubert,Classical
3500,"L'orfeo, Act 3, Sinfonia (Orchestra)",Claudio Monteverdi,Classical
3501,"Quintet for Horn, Violin, 2 Violas, and Cello ...",Wolfgang Amadeus Mozart,Classical


In [19]:
query = '''
SELECT b.name AS genre_name, SUM(a.unitprice) AS TOT_PRICE
FROM tracks AS a
INNER JOIN genres AS b ON a.GenreId=b.GenreId
GROUP BY b.name
ORDER BY TOT_PRICE DESC
LIMIT 10;
'''
# Define una consulta SQL: La variable query contiene una cadena con una consulta SQL que selecciona los géneros de música y la suma de 
#los precios unitarios de las pistas para cada género. Luego, ordena los géneros por el total de precios en orden descendente y limita
#el resultado a los 10 géneros con los mayores totales. 
sql_query(query) 

,genre_name,TOT_PRICE
0,Rock,1284.03
1,Latin,573.21
2,Metal,370.26
3,Alternative & Punk,328.68
4,TV Shows,185.07
5,Jazz,128.70
6,Drama,127.36
7,Blues,80.19
8,Classical,73.26
9,R&B/Soul,60.39


![imagen](./img/chinook_data_model.png)

In [20]:

query = '''
SELECT a.trackid, a.name, a.composer, b.invoicelineid, b.invoiceid
FROM tracks AS a
INNER JOIN invoice_items AS b
ON a.trackid = b.trackid
;
'''
# SELECT a.trackid, a.name, a.composer, b.invoicelineid, b.invoiceid: Selecciona los campos trackid, name, composer de la tabla tracks
#(con alias a), y invoicelineid, invoiceid de la tabla invoice_items (con alias b).
# FROM tracks AS a: Especifica que los datos deben ser seleccionados de la tabla tracks con el alias a.
# INNER JOIN invoice_items AS b: Realiza una unión interna entre la tabla tracks y la tabla invoice_items, asignando el alias b a
#invoice_items.
# ON a.trackid = b.trackid: Define la condición de la unión, donde el campo trackid de tracks debe coincidir con el campo trackid de 
#invoice_items.
sql_query(query) 

,TrackId,Name,Composer,InvoiceLineId,InvoiceId
0,2,Balls to the Wall,None,1,1
1,4,Restless and Wild,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",2,1
2,6,Put The Finger On You,"Angus Young, Malcolm Young, Brian Johnson",3,2
3,8,Inject The Venom,"Angus Young, Malcolm Young, Brian Johnson",4,2
4,10,Evil Walks,"Angus Young, Malcolm Young, Brian Johnson",5,2
...,...,...,...,...,...
2235,3136,Looking For Love,Sykes,2236,411
2236,3145,Sweet Lady Luck,Vandenberg,2237,411
2237,3154,Feirinha da Pavuna/Luz do Repente/Bagaço da La...,"Arlindo Cruz/Franco/Marquinhos PQD/Negro, Jove...",2238,411
2238,3163,Samba pras moças,Grazielle/Roque Ferreira,2239,411


In [21]:

query = '''
SELECT name AS track_title, albumid
FROM tracks;
'''

sql_query(query)

,track_title,AlbumId
0,For Those About To Rock (We Salute You),1
1,Balls to the Wall,2
2,Fast As a Shark,3
3,Restless and Wild,3
4,Princess of the Dawn,3
...,...,...
3498,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,343
3499,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",344
3500,"L'orfeo, Act 3, Sinfonia (Orchestra)",345
3501,"Quintet for Horn, Violin, 2 Violas, and Cello ...",346


In [23]:
# 1- Definición de la consulta SQL:
# - 'SELECT a.name AS track_title , b.title AS album_title': Selecciona el nombre de la pista (a.name) como track_title y el título del 
#álbum (b.title) como album_title.
# - 'FROM tracks AS a': Especifica que los datos deben ser seleccionados de la tabla tracks con el alias a.
# - 'LEFT JOIN albums AS b ON a.albumid = b.albumid': Realiza una unión externa izquierda entre la tabla tracks y la tabla albums, 
#asignando el alias b a albums, y uniendo las tablas en el campo albumid.

# 2.- Ejecución de la consulta SQL: 
# - Esta línea llama a la función sql_query con el argumento query y almacena el resultado en el DataFrame df_lj.
# - La función sql_query no está definida en el fragmento de código proporcionado, pero se puede inferir que esta función ejecuta la
#consulta SQL en una base de datos y devuelve un DataFrame. 

# 3.- Filtrado del DataFrame: 
# - Filtra el DataFrame df_lj para mostrar únicamente las filas donde el valor en la columna album_title es NaN (indicando que no hay
#un álbum asociado con la pista). 

# 4.- Resultado final: 
# - Esta línea parece estar incompleta o incorrecta, ya que no está haciendo nada con el DataFrame filtrado. Podría ser un error 
#tipográfico.

#LEFT JOIN
query = '''
SELECT a.name AS track_title , b.title AS album_title
FROM tracks AS a
LEFT JOIN albums AS b ON a.albumid = b.albumid
'''

df_lj = sql_query(query)
df_lj[df_lj['album_title'].isna()]
df 

# 1.- Define una consulta SQL: La variable query contiene una cadena con una consulta SQL que selecciona el título de la pista y el título
#del álbum de las tablas tracks y albums utilizando una unión externa izquierda (LEFT JOIN).

#2.- Ejecuta la consulta SQL: La función sql_query(query) se llama con la consulta SQL como argumento. Esta función ejecuta la consulta
#en una base de datos y devuelve el resultado en un DataFrame.

#3.- Filtra los resultados: El DataFrame resultante (df_lj) se filtra para mostrar únicamente las filas donde el título del álbum 
#(album_title) es NaN (es decir, aquellas pistas que no tienen un álbum asociado). 

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99
...,...,...,...,...,...,...,...,...,...
3498,3499,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,343,2,24,None,286741,4718950,0.99
3499,3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",344,2,24,Franz Schubert,139200,2283131,0.99
3500,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",345,2,24,Claudio Monteverdi,66639,1189062,0.99
3501,3502,"Quintet for Horn, Violin, 2 Violas, and Cello ...",346,2,24,Wolfgang Amadeus Mozart,221331,3665114,0.99


In [24]:
# 1.- Define una consulta SQL: La variable query contiene una cadena con una consulta SQL que selecciona todas las columnas de las tablas
#invoice_items y tracks utilizando una unión externa izquierda (LEFT JOIN) y limita el resultado a las primeras 5 filas. 

# 1.- Definición de la consulta SQL: 
# - SELECT *: Selecciona todas las columnas de las tablas especificadas.
# - FROM invoice_items AS a: Especifica que los datos deben ser seleccionados de la tabla invoice_items con el alias a.
# - LEFT JOIN tracks AS b ON a.trackid = b.trackid: Realiza una unión externa izquierda entre la tabla invoice_items (con alias a) y 
#la tabla tracks (con alias b), uniendo las tablas en el campo trackid.
# - LIMIT 5: Limita el resultado a las primeras 5 filas. 

# 2.- Ejecución de la consulta SQL: 
# - Esta línea llama a la función sql_query con el argumento query.
# - La función sql_query no está definida en el fragmento de código proporcionado, pero se puede inferir que esta función probablemente 
#ejecuta la consulta SQL en una base de datos y devuelve el resultado.

query = '''
SELECT * 
FROM invoice_items AS a
LEFT JOIN tracks AS b ON a.trackid = b.trackid
LIMIT 5;
'''

sql_query(query) 

,InvoiceLineId,InvoiceId,TrackId,UnitPrice,Quantity,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,1,2,0.99,1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
1,2,1,4,0.99,1,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
2,3,2,6,0.99,1,6,Put The Finger On You,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",205662,6713451,0.99
3,4,2,8,0.99,1,8,Inject The Venom,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",210834,6852860,0.99
4,5,2,10,0.99,1,10,Evil Walks,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",263497,8611245,0.99


In [25]:
# El código realiza una consulta SQL que:
# 1 - Primer SELECT: 
# - Este primer SELECT realiza una unión externa izquierda (LEFT JOIN) entre invoice_items (con alias a) y tracks (con alias b), uniendo
# las tablas en el campo trackid.
# - Esto significa que todos los registros de invoice_items se seleccionarán, junto con los registros correspondientes de tracks. Si no 
# hay una coincidencia en tracks, las columnas de tracks serán NULL.

# 2 - UNION: 
# - La cláusula UNION combina los resultados de dos consultas SELECT.
# - El UNION elimina las filas duplicadas por defecto. Si deseas incluir duplicados, debes usar UNION ALL.

# 3 - Segundo SELECT: 
# - Este segundo SELECT realiza una unión externa izquierda (LEFT JOIN) entre la tabla tracks (con alias a) y la tabla invoice_items 
# (con alias b), uniendo las tablas en el campo trackid.
# - Esto significa que todos los registros de tracks se seleccionarán, junto con los registros correspondientes de invoice_items. Si no 
# hay una coincidencia en invoice_items, las columnas de invoice_items serán NULL.

# 4 - LIMIT: 
# - La cláusula LIMIT especifica que solo se deben devolver las primeras 5 filas del resultado final.
# - Nota: En algunos sistemas de bases de datos, LIMIT no se puede utilizar directamente con UNION sin subconsultas. 

query = '''
SELECT * 
FROM invoice_items AS a
LEFT JOIN tracks AS b
ON a.trackid = b.trackid

UNION

SELECT *
FROM tracks AS a
LEFT JOIN invoice_items AS b
ON a.trackid =b.trackid 
LIMIT 5;
'''

sql_query(query) 

,InvoiceLineId,InvoiceId,TrackId,UnitPrice,Quantity,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,1,2,0.99,1,2,Balls to the Wall,2,2.00,1,None,342562,5510424.00,0.99
1,1,For Those About To Rock (We Salute You),1,1.00,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99,579,108,1,0.99,1.00
2,2,1,4,0.99,1,4,Restless and Wild,3,2.00,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779.00,0.99
3,2,Balls to the Wall,2,2.00,1,None,342562,5510424,0.99,1,1,2,0.99,1.00
4,2,Balls to the Wall,2,2.00,1,None,342562,5510424,0.99,1154,214,2,0.99,1.00


In [26]:
#FULL JOIN
#FULL JOIN no está soportado en SQLite
#hay que usar LEFT JOIN + RIGHT JOIN(inversa de LEFT JOIN) como una solución provisional

query = '''
SELECT * 
FROM invoice_items AS a
LEFT JOIN tracks AS b
ON a.trackid = b.trackid

UNION ALL

SELECT *
FROM tracks AS a
LEFT JOIN invoice_items AS b 
ON a.trackid =b.trackid;
'''

sql_query(query) 

# 1 - Primer SELECT: 
# - Este primer SELECT realiza una unión externa izquierda (LEFT JOIN) entre invoice_items (con alias a) y tracks (con alias b), uniendo
# las tablas en el campo trackid.
# - Esto significa que todos los registros de invoice_items se seleccionarán, junto con los registros correspondientes de tracks. Si no 
# hay una coincidencia en tracks, las columnas de tracks serán NULL. 

# 2 - UNION ALL: 
# - La cláusula UNION ALL combina los resultados de dos consultas SELECT.
# - A diferencia de UNION, UNION ALL no elimina las filas duplicadas. Esto significa que todas las filas del primer SELECT y del segundo
# SELECT se incluirán en el resultado final, incluso si hay duplicados. 

# 3 - Segundo SELECT: 
# - Este segundo SELECT realiza una unión externa izquierda (LEFT JOIN) entre tracks (con alias a) e invoice_items (con alias b), uniendo
# las tablas en el campo trackid.
# - Esto significa que todos los registros de tracks se seleccionarán, junto con los registros correspondientes de invoice_items. Si no 
# hay una coincidencia en invoice_items, las columnas de invoice_items serán NULL. 

,InvoiceLineId,InvoiceId,TrackId,UnitPrice,Quantity,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,1,2,0.99,1,2,Balls to the Wall,2,2.00,1.0,None,342562.0,5510424.00,0.99
1,2,1,4,0.99,1,4,Restless and Wild,3,2.00,1.0,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051.0,4331779.00,0.99
2,3,2,6,0.99,1,6,Put The Finger On You,1,1.00,1.0,"Angus Young, Malcolm Young, Brian Johnson",205662.0,6713451.00,0.99
3,4,2,8,0.99,1,8,Inject The Venom,1,1.00,1.0,"Angus Young, Malcolm Young, Brian Johnson",210834.0,6852860.00,0.99
4,5,2,10,0.99,1,10,Evil Walks,1,1.00,1.0,"Angus Young, Malcolm Young, Brian Johnson",263497.0,8611245.00,0.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5994,3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",344,2.00,24,Franz Schubert,139200,2283131,0.99,578.0,108,3500.0,0.99,1.00
5995,3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",344,2.00,24,Franz Schubert,139200,2283131,0.99,1727.0,319,3500.0,0.99,1.00
5996,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",345,2.00,24,Claudio Monteverdi,66639,1189062,0.99,NaN,None,NaN,NaN,NaN
5997,3502,"Quintet for Horn, Violin, 2 Violas, and Cello ...",346,2.00,24,Wolfgang Amadeus Mozart,221331,3665114,0.99,NaN,None,NaN,NaN,NaN


### 3.10 VIEW y DROP
En ocasiones resulta útil **dividir nuestras queries en varios pasos**, y para ello utilizamos la sentencia `VIEW`. Con `VIEW` creamos una *vista* de una tabla, es decir, obtenemos una tabla temporal que usaremos después.

Estas vistas se guardarán en el archivo *.db*, por lo que tendrás que tener cuidado de no almacenar muchas tablas poco útiles.

Primero, veamos cómo acceder a todas las vistas de la base de datos

In [28]:
# 1 - Obtener los nombres de las vistas:
# - La consulta 'SELECT name FROM sqlite_master WHERE type='view'' obtiene una lista de todas las vistas en la base de datos, extrayendo el campo name de la tabla sqlite_master
#donde el type de dato es 'view'. 
# - Esta consulta devuelve un cursor que puede ser iterado para acceder a los nombres de las vistas.

# 2 - Imprimir los datos del campo 'name' que sean del tipo 'view':
# - El bucle for name in res: recorre cada fila del resultado. 
# - 'print(name[0])' imprime el primer elemento de cada fila, que es el nombre de la vista. 

# EXPLICACION DETALLADA 
# 1 - Ejecución de la consulta SQL: 
# - 'crsr.execute(...)' ejecuta la consulta SQL utilizando el cursor crsr.
# - 'crsr' es una instancia de sqlite3.Cursor, que se usa para interactuar con la base de datos SQLite.
# - El método 'execute' de sqlite3.Cursor ejecuta la consulta SQL y devuelve el cursor mismo (crsr).

# 2 - Iterar sobre los resultados: 
# - 'res' es un objeto sqlite3.Cursor, que es un iterador sobre los resultados de la consulta SQL.
# - En cada iteración, name es una tupla que representa una fila del conjunto de resultados. 


crsr = connection.cursor() # SE CREA EL CURSOR 'crsr' 

res = crsr.execute("SELECT name FROM sqlite_master WHERE type='view'") # SE EJECUTA LA CONSULTA PARA LUEGO ASIGNAR EL RESULTADO DE DICHA CONSULTA A LA VARIABLE 'res' 
for name in res:
    print(name[0]) 

Creamos dos vistas nuevas

In [29]:
# El código realiza los siguientes pasos:

# 1 - Crea dos vistas en la base de datos:
# - La primera vista se llama name_composer y selecciona las columnas name, composer y genreid de la tabla tracks.
# - La segunda vista se llama genre y selecciona las columnas name y genreid de la tabla genres, filtrando los resultados para que solo incluyan los géneros cuyos nombres
#comiencen con la letra "b".

# 2 - Ejecuta estas consultas para crear las vistas en la base de datos:
# - Utiliza el cursor (crsr) para ejecutar las consultas query1 y query2.

# 3 - Ejecuta una consulta SQL no especificada:
# - La variable query se menciona en el último paso, pero su contenido no está definido en el fragmento de código proporcionado. La función sql_query(query) ejecuta esta consulta 
#y devuelve los resultados.

query1 = '''
CREATE VIEW name_composer AS
SELECT name, composer, genreid
FROM tracks;
'''

query2 = '''
CREATE VIEW genre AS
SELECT name, genreid
FROM genres
WHERE name LIKE "b%";
'''

crsr.execute(query1)
crsr.execute(query2)

sql_query(query) 

# EXPLICACIÓN ESPECIFICA 

# 1 - Creación de la primera vista (name_composer), VISTA ES UN OBJETO VIRTUAL QUE NO OCUPA MEMORIA Y QUE NO AFECTA LA TABLA ORIGINAL
# - Esta consulta crea una vista llamada 'name_composer'. LA VISTA SIRVE PARA MOSTRAR DATOS AL USUARIO 
# - La vista contiene las columnas 'name', 'composer' y 'genreid' de la tabla 'tracks'.

# 2 - Creación de la segunda vista (genre) 
# - Esta consulta crea una vista llamada 'genre'.
# - La vista contiene las columnas 'name' y 'genreid' de la tabla 'genres'.
# - Solo incluye las filas donde el atributo 'name' de la tabla'genres' comienza con la letra "b".

# 3 - Ejecución de las consultas para crear las vistas 
# - Estas líneas ejecutan las consultas query1 y query2 para crear las vistas 'name_composer' y 'genre' en la base de datos.

# 4 - Ejecución de una consulta SQL adicional 
# - Esta línea ejecuta una consulta SQL definida en la variable 'query' utilizando la función 'sql_query'. Dado que 'query' no está definido en el fragmento proporcionado, se asume 
#que está definido en otro lugar del código.

,InvoiceLineId,InvoiceId,TrackId,UnitPrice,Quantity,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,1,2,0.99,1,2,Balls to the Wall,2,2.00,1.0,None,342562.0,5510424.00,0.99
1,2,1,4,0.99,1,4,Restless and Wild,3,2.00,1.0,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051.0,4331779.00,0.99
2,3,2,6,0.99,1,6,Put The Finger On You,1,1.00,1.0,"Angus Young, Malcolm Young, Brian Johnson",205662.0,6713451.00,0.99
3,4,2,8,0.99,1,8,Inject The Venom,1,1.00,1.0,"Angus Young, Malcolm Young, Brian Johnson",210834.0,6852860.00,0.99
4,5,2,10,0.99,1,10,Evil Walks,1,1.00,1.0,"Angus Young, Malcolm Young, Brian Johnson",263497.0,8611245.00,0.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5994,3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",344,2.00,24,Franz Schubert,139200,2283131,0.99,578.0,108,3500.0,0.99,1.00
5995,3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",344,2.00,24,Franz Schubert,139200,2283131,0.99,1727.0,319,3500.0,0.99,1.00
5996,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",345,2.00,24,Claudio Monteverdi,66639,1189062,0.99,NaN,None,NaN,NaN,NaN
5997,3502,"Quintet for Horn, Violin, 2 Violas, and Cello ...",346,2.00,24,Wolfgang Amadeus Mozart,221331,3665114,0.99,NaN,None,NaN,NaN,NaN


Comprueba que ahora tienes vistas nuevas en tu base de datos

In [30]:
res = crsr.execute("SELECT name FROM sqlite_master WHERE type='view';")
for name in res:
    print(name[0])

# EXPLICACÍON DETALLADA 

# 1 - Ejecutar una consulta SQL: 
# - 'crsr.execute("SELECT name FROM sqlite_master WHERE type='view';")' ejecuta una consulta SQL que selecciona los nombres (name) de todas las vistas en la base de datos.
# - 'sqlite_master' es una tabla especial en SQLite que almacena información sobre todas las tablas, índices, vistas y triggers que existen en la base de datos.
# - 'WHERE type='view'' filtra los resultados para incluir solo las entradas que son de tipo 'view'. 

# 2 - Iterar sobre los resultados: 
# - 'for name in res': itera sobre el conjunto de resultados devuelto por la consulta.
# - 'print(name[0])' imprime el nombre de cada vista. name es una tupla donde el primer elemento (name[0]) es el nombre de la vista. 

# Tipo de Variable
# - La variable res es de tipo sqlite3.Cursor, que es un iterador de resultados devueltos por la consulta SQL.

# Comportamiento del Cursor
# - Iterador: El cursor actúa como un iterador, lo que significa que puedes recorrerlo en un bucle for para acceder a cada fila de resultados.
# - Métodos de Cursor: Además de ser un iterador, el cursor proporciona varios métodos para trabajar con los resultados, como fetchone(), fetchall(), y fetchmany(size).

name_composer
genre


Puedes borrar las vistas mediante `DROP VIEW`. Prueba a borrarlas, comprueba que ya no existen y vuelve a crearlas.

**NOTA**: `DROP` también lo podrás usar como `DROP TABLE`. Nos cargaríamos una tabla entera de la base de datos y ya no sería recuperable, el *ctrl + Z* no te va a salvar. así que mucho cuidado con esta sentencia.

In [31]:
# Este código elimina una vista llamada 'genre' de la base de datos SQLite si esta existe 

query2_ = '''
DROP VIEW IF EXISTS genre;
'''

crsr.execute(query2_) 

# 1 - Definición de la consulta SQL: 
# - query2_ es una cadena que contiene la consulta SQL.
# - DROP VIEW IF EXISTS genre; es la instrucción SQL que elimina una vista llamada genre si esta existe en la base de datos.
# - DROP VIEW es una instrucción SQL utilizada para eliminar una vista existente.
# - IF EXISTS es una cláusula opcional que evita errores en caso de que la vista no exista. Si la vista genre no existe, no se producirá un error, simplemente no se hará nada.

# 2 - Ejecutar la consulta SQL: 
# - 'crsr.execute(query2_)' ejecuta la consulta SQL definida en query2_ utilizando el cursor crsr.
# - Esta línea de código envía la instrucción 'DROP VIEW IF EXISTS genre'; a la base de datos para su ejecución. 

#   + 'DROP VIEW IF EXISTS genre'; es la instrucción SQL que elimina una vista llamada genre si esta existe en la base de datos.

# - Si la vista 'genre' existe, se eliminará de la base de datos. Si no existe, no ocurrirá nada y no se generará un error. 

Ahora que ya tenemos nuestras vistas, vamos a realizar otra operación con ambas tablas. En este caso, aplicamos un `JOIN`

Fíjate que la salida de esta query es la misma que la del primer `JOIN` de este Notebook, pero en este caso, hecho en dos pasos.

In [32]:
# Este código realiza una consulta SQL que selecciona datos de dos vistas (name_composer y genres), las une en base al campo 'genreid' y luego ejecuta la consulta para obtener
#un DataFrame con los resultados.

query = '''
SELECT a.name, a.composer, b.name
FROM name_composer AS a
INNER JOIN genres AS b ON a.genreid = b.genreid;
'''

df = sql_query(query) 

In [35]:
# se utiliza en el contexto de la biblioteca pandas de Python. Este código guarda un DataFrame (una estructura de datos bidimensional, 
 #similar a una tabla, que puede contener datos de diferentes tipos) en un archivo CSV (valores separados por comas).

df.to_csv('my_dataset.csv') 

# Aquí hay una explicación detallada de cada parte del código:

# df: Es el DataFrame que deseas guardar. Supongo que df ha sido definido previamente en tu código y contiene los datos que quieres
#exportar. 

# .to_csv: Es un método de los DataFrames de pandas que permite exportar los datos del DataFrame a un archivo CSV. 

# 'my_dataset.csv': Es el nombre del archivo CSV en el que se guardarán los datos. Este archivo se creará en el directorio actual donde 
#se está ejecutando el script, a menos que se especifique una ruta diferente. 

# Por lo tanto, el código df.to_csv('my_dataset.csv') guarda el contenido del DataFrame df en un archivo llamado my_dataset.csv en el
#directorio actual. 

In [37]:
query1_ = '''
DROP VIEW IF EXISTS name_composer;
'''
crsr.execute(query1_) 

# DESCRIPCION EN DETALLE 

# 1.- Consulta SQL: 
# - DROP VIEW: Este comando se utiliza para eliminar una vista existente en la base de datos.
# - IF EXISTS: Este modificador es opcional y se utiliza para evitar errores en caso de que la vista que se desea eliminar no exista. 
#Si la vista no existe, no se hará nada y no se generará un error.
# - name_composer: Es el nombre de la vista que se desea eliminar.

# Por lo tanto, esta consulta elimina la vista llamada name_composer si existe en la base de datos.

# 2.- Ejecución en Python: 
# - query1_: Es una variable de cadena que contiene la consulta SQL.
# - crsr: Es el cursor del objeto de conexión a la base de datos. Este cursor se utiliza para ejecutar comandos SQL y recuperar datos
#de la base de datos.
# - crsr.execute(query1_): Este método del cursor ejecuta la consulta SQL almacenada en query1_.

# En resumen, el código completo tiene como objetivo eliminar la vista llamada name_composer de la base de datos si esta existe. 

## 4. Errores

<table align="left">
 <tr><td width="80"><img src="./img/error.png" style="width:auto;height:auto"></td>
     <td style="text-align:left">
         <h3>ERRORES Acceso a base de datos que no existe</h3>
         
 </td></tr>
</table>

¡Cuidado! No da error, sino que crea una base de datos vacía

In [33]:
# Conectamos con la base de datos chinuk.db
connection = sqlite3.connect("chinuk.db")

crsr = connection.cursor()

<table align="left">
 <tr><td width="80"><img src="./img/error.png" style="width:auto;height:auto"></td>
     <td style="text-align:left">
         <h3>ERRORES en la query</h3>
         
 </td></tr>
</table>

Los errores son todos del mismo tipo: `OperationalError`. Esto nos indica que lo da el propio SQL, no es un error del intérprete de Python como tal. Ahora bien, aunque lo de SQL, devuelve un descriptivo con el tipo de error, que está bastante bien para ver por donde van los tiros.

In [34]:
# Conectamos con la base de datos chinook.db
connection = sqlite3.connect("chinook.db")

# Obtenemos un cursor que utilizaremos para hacer las queries
crsr = connection.cursor()

In [35]:
query = '''
SELECT * 
FROM tracks
'''

sql_query(query)

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99
...,...,...,...,...,...,...,...,...,...
3498,3499,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,343,2,24,None,286741,4718950,0.99
3499,3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",344,2,24,Franz Schubert,139200,2283131,0.99
3500,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",345,2,24,Claudio Monteverdi,66639,1189062,0.99
3501,3502,"Quintet for Horn, Violin, 2 Violas, and Cello ...",346,2,24,Wolfgang Amadeus Mozart,221331,3665114,0.99


In [41]:
query = '''
SELECT * 
FROM tracksss
'''

sql_query(query)

OperationalError: no such table: tracksss

In [50]:
query = '''
SELECT campo1, campo2
FROM tracks
'''

sql_query(query)

OperationalError: no such column: campo1

## 5. Resumen
**SQL es el estándar que se utiliza en el acceso a las BBDD relacionales**, por lo que en prácticamente cualquier empresa te vas a encontrar una BD de este tipo. La sintaxis es:

```SQL
SELECT campo1, campo2, campo3...
FROM tabla
WHERE condicionales
```

Y podrás realizar las siguientes acciones:
1. **Filtrar columnas**: mediante `SELECT`
2. **Renombrar campos**: mediante `SELECT campo as nuevo_nombre`
3. **Eliminar duplicados**: mediante `DISTINCT`
4. **Limitar el número de registros**: `LIMIT`
5. **Filtrar filas**: `WHERE`
6. **Ordenar una tabla**: `ORDER BY campo1, campo2` y `DESC`
7. **Agregar información**: sacar un KPI como un máximo o un conteo: `MAX`,`COUNT`,`AVG`...
8. **Agregado a nivel grupo**: con `GROUP BY`. Se calculan las funciones de agregación para cada grupo de una columna.
9. **Juntar datos**: mediante `JOIN`. Hay de varios tipos: `LEFT`, `RIGHT`, `FULL`, `INNER` y `OUTER`.
10. **Tablas temporales**: con `VIEW`
11. **Eliminar tablas o vistas**: con `DROP`

Y si te manejas de maravilla con la librería pandas, **siempre puedes obtener tus datos mediante una query con sqlite3, y tratarlos posteriormente en Python, mediante las funciones de pandas.**